In [1]:

# Data handling
import pandas as pd
import numpy as np

# Data visualisation
import matplotlib.pyplot as plt
import seaborn as sns

#Clustering & Classification
from sklearn.cluster import KMeans
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.compose import ColumnTransformer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from sklearn.model_selection import train_test_split

import random

# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

In [2]:
#import testing and training datasets
df_train = pd.read_excel('BankChurnersTrain.xls')
df_test = pd.read_excel('BankChurnersTest.xls')

FileNotFoundError: [Errno 2] No such file or directory: 'BankChurnersTrain.xls'

In [ ]:
df_combined = pd.concat([df_test, df_train])

In [ ]:
df_combined.head()

# Data Exploration 

In [ ]:
df_combined.dtypes

In [ ]:
df_combined.describe()

In [ ]:
df_combined.isnull().sum()

In [ ]:
#Data Cleaning

In [ ]:
#Handling missing values and input with mean
df_combined.fillna(df_combined.mean(), inplace=True)

In [ ]:
# Drop irrelevant columns
drop_columns = ['CCNum', 'Trans_date_Time', 'Surname']
df_combined=df_test.drop(labels=drop_columns, axis =1)

In [ ]:
object_columns = ['Attrition_Flag', 'Gender', 'Geography', 'Education_Level', 'Income_Category', 'Card_Category']

In [ ]:
# Define mappings for object to int conversion
mapping = {
    'Attrition_Flag': {'Attrited Customer': 1, 'Existing Customer': 0},
    'Gender': {'F': 0, 'M': 1},
    'Geography': {'France': 0, 'Germany': 1, 'Spain': 2},
    'Education_Level': {'Unknown': 0, 'Uneducated': 1, 'High School': 2, 'College': 3, 'Graduate': 4, 'Post-Graduate': 5, 'Doctorate': 6},
    'Income_Category': {'Unknown': 0, 'Less than $40K': 1, '$40K - $60K': 2, '$60K - $80K': 3, '$80K - $120K': 4, '$120K +': 5},
    'Card_Category': {'Blue': 0, 'Silver': 1, 'Gold': 2, 'Platinum': 3}
}

In [ ]:
for column in object_columns:
    df_combined[column] = df_combined[column].map(mapping[column]).astype(int)

In [ ]:
print(df_combined.dtypes)

In [ ]:
df_combined.head()

In [ ]:
columns = ['Age',
       'Gender', 'Geography', 'Tenure', 'Education_Level',
       'Income_Category', 'Card_Category', 'NumOfProducts', 'HasCrCard',
       'IsActiveMember', 'EstimatedSalary', 'is_fraud', 'Exited']

for i in columns:
    plt.figure()
    sns.distplot(df_combined[i]);

In [ ]:
#Churn Frequency by country
pd.crosstab(df_combined.Geography, df_combined.Attrition_Flag).plot(kind='bar')
plt.title('Customer Attrition Frequency by Country')
plt.xlabel('Country')
plt.ylabel('Attrition Frequency')

In [ ]:
columns = ['Age',
       'Gender', 'Geography', 'Tenure', 'Education_Level',
       'Income_Category', 'Card_Category', 'NumOfProducts', 'HasCrCard',
       'IsActiveMember', 'EstimatedSalary', 'is_fraud', 'Exited']

for i in columns:
    plt.figure()
    pd.crosstab(df_combined[i], df_combined.Attrition_Flag).plot(kind='bar');

In [ ]:
df_combined['Attrition_Flag'].value_counts()

In [ ]:
sns.histplot(df_combined['Attrition_Flag'])

In [ ]:
attrited_customers = df_combined[df_combined['Attrition_Flag']== 1]
attrited_customers

In [ ]:
# Create an instance of the Random Forest classifier
rf_classifier = RandomForestClassifier()

In [ ]:
# Extract the input features from the training data
X_train = df_combined.drop('Attrition_Flag', axis=1)

In [ ]:
# Extract the target labels from the training data
y_train = df_combined['Attrition_Flag']

In [ ]:
# Train the classifier on the training data
rf_classifier.fit(X_train, y_train)

In [ ]:
# Target variable
X = df_combined.drop('Attrition_Flag', axis=1)
y = df_combined['Attrition_Flag']

In [ ]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Make predictions on the test data
y_pred = rf_classifier.predict(X_test)

In [ ]:
# Visualise the feature importance
d = {'Columns': X_train.columns, 'Feature Importance': rf_classifier.feature_importances_}
high_attrition = pd.DataFrame(d)
high_attrition_sorted = high_attrition.sort_values(by='Feature Importance', ascending = False)
high_attrition_sorted
high_attrition_sorted.style.background_gradient(cmap='Blues')

In [ ]:
high_attrition_attributes = ['Total_Trans_Ct', 'Total_Trans_Amt', 'Credit_Limit', 'EstimatedSalary', 'Age', 'CreditScore', 'Balance', 'Tenure']

In [ ]:
df_combined.columns

In [ ]:
# Filter the dataset based on the high attrition attributes
high_attrition_df = df_combined[high_attrition_attributes]
high_attrition_df.head()

In [ ]:
#create customer profiles based on high attrition attributes
number_of_clusters = 3
customer_profiles = {}
for i in range (0, len(high_attrition_attributes), number_of_clusters):
    attribute_list = []
    for attribute in high_attrition_attributes[i:i + number_of_clusters]:
        attribute_list.append(attribute)
        customer_profiles[i] = attribute_list

# Generate a new attribute
new_attribute = random.choice(high_attrition_attributes)

# Check if the new attribute is already in the list
while new_attribute in customer_profiles[6]:
    new_attribute = random.choice(high_attrition_attributes)

# Add the new attribute to the list
customer_profiles[6].append(new_attribute)
        
customer_profiles

In [ ]:
attrition_rate = df_combined['Attrition_Flag'].value_counts(normalize=True)[1]
attrition_rate

In [ ]:
attrition_rates= []

In [ ]:
for profile, attributes in customer_profiles.items():
    profile_data = df_combined.loc[df_combined[attributes].all(axis=1)]
    profile_attrition_rate = profile_data['Attrition_Flag'].value_counts(normalize=True).get(1, 0)
    attrition_rates.append(profile_attrition_rate)
    
attrition_rates

In [ ]:
#calculate the churn rate for each customer
attrition_rate = df_combined["Attrition_Flag"].mean()
attrition_rate

In [ ]:
df_combined["Attrition_Risk"] = np.where(df_combined["Attrition_Flag"] == 1, "High", "Low")

In [ ]:
#customers most likely to churn
df_combined = df_combined.sort_values("Attrition_Risk", ascending = False)

In [ ]:
df_combined.head()

In [ ]:
print(f"Profile {profile}:")
print(f"Attributes: {attributes}")
print(f"Attrition Rate: {profile_attrition_rate:.2%}")

In [ ]:
# Plotting the attrition rates
plt.figure(figsize=(8, 6))
plt.bar(range(len(attrition_rates)), attrition_rates)
plt.xlabel('Profile')
plt.ylabel('Attrition Rate')
plt.title('Attrition Rate by Customer Profiles')
plt.xticks(range(len(attrition_rates)), [f"Profile {profile}" for profile in customer_profiles.keys()])
plt.show()

In [ ]:
# Sort the dataframe by highest risk of attrition in descending order
high_attrition_customers_df = df_combined.sort_values(by="Attrition_Risk", ascending=True)

# Print the sorted dataframe
print(high_attrition_customers_df)

In [ ]:
# Visualise clusters - Create a KMeans object with 3 clusters
kmeans = KMeans(n_clusters = 3, n_init = 20)

In [ ]:
# Fit the KMeans model to the high attrition data
kmeans.fit(high_attrition_df[high_attrition_attributes])

In [ ]:
# Get the cluster labels for each data point
cluster_labels = kmeans.labels_
cluster_labels

In [ ]:
# Create a scatter plot of the data points, coloured by cluster label
plt.scatter(high_attrition_df['Total_Trans_Ct'], high_attrition_df['Total_Trans_Amt'], c=cluster_labels)
plt.title('Customer Churn Clusters')
plt.xlabel('Total_Trans_Ct')
plt.ylabel('Total_Trans_Amt')
plt.show()

In [ ]:
# Churn Scrutiny

In [ ]:
# Plot the relationship between balance and tenure.
plt.scatter(df_train["Balance"], df_train["Tenure"])
plt.title("Balance vs. Tenure")
plt.xlabel("Balance")
plt.ylabel("Tenure")
plt.show()

In [ ]:
# Calculate the correlation coefficient between balance and tenure.
correlation = np.corrcoef(df_combined["Balance"], df_combined["Tenure"])[0, 1]
print("The correlation coefficient between balance and tenure is", correlation.round(4))

In [ ]:
df_train.corr(method="pearson")

In [ ]:
correlation_matrix = df_train.corr()

In [ ]:
correlation_matrix.describe()

In [ ]:
#threshold for strong relationships
threshold = 0.7

In [ ]:
strong_relationships = []
# Filter the correlation matrix to identify strongly correlated attributes
strong_correlations = correlation_matrix[abs(correlation_matrix) >= 0.7]
strong_correlations = strong_correlations.unstack().dropna().sort_values(ascending=False)

In [ ]:
print("Strongly correlated attribute pairs:")
print(strong_correlations)

In [ ]:
plt.figure(figsize=(10,15))
sns.heatmap(df_train.corr(), annot= True, cmap = 'coolwarm')
plt.title("Correlation Matrix for Attributes")

In [ ]:
for i in range (len(correlation_matrix.columns)):
    for j in range(i+1, len(correlation_matrix.columns)):
        if abs(correlation_matrix.iloc[i,j]) >= threshold:
            strong_relationships.append((correlation_matrix.columns[i], correlation_matrix.columns[j]))

In [ ]:
for attribute_pair in strong_relationships:
    print("Strong relationship between", attribute_pair[0], "and", attribute_pair[1])

In [ ]:
#Sort the high-value targets by balance
high_value_attributes = correlation_matrix["Balance"].sort_values(ascending=False).index[:5]

In [ ]:
#Print the high-value attributes
print(high_value_attributes)

In [ ]:
#Identify the high-value targets
high_value_targets = df_train[df_train[high_value_attributes].max(axis=1) > 175000]

In [ ]:
# Calculate and print the evaluation metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

In [ ]:
# Generate the classification report
report = classification_report(y_test, y_pred)
print("Classification Report:")
print(report)